# Calculate well-averages for CellProfiler measurements

In [1]:
import os
import pandas as pd

#print(os.getcwd())
#print(os.listdir())
#print(os.listdir("work"))

workdir = "/home/hajaalin/tmp"
filename = "MyExpt_Cells.csv"
basename, extension = os.path.splitext(filename)

df = pd.read_csv(os.path.join(workdir, filename))
df.head()

,ImageNumber,ObjectNumber,Metadata_ChannelNumber,Metadata_FileLocation,Metadata_Frame,Metadata_Plate,Metadata_Series,Metadata_Site,Metadata_Well,AreaShape_Area,...,Texture_SumEntropy_Red_5_02,Texture_SumEntropy_Red_5_03,Texture_SumVariance_Red_5_00,Texture_SumVariance_Red_5_01,Texture_SumVariance_Red_5_02,Texture_SumVariance_Red_5_03,Texture_Variance_Red_5_00,Texture_Variance_Red_5_01,Texture_Variance_Red_5_02,Texture_Variance_Red_5_03
0,79,1,NaN,NaN,0,080119-testi1,0,3,E04,2008,...,0.065773,0.076868,0.004032,0.008097,0.007731,0.009323,0.002020,0.004065,0.003881,0.004684
1,79,2,NaN,NaN,0,080119-testi1,0,3,E04,3006,...,1.822146,1.921561,0.773529,0.787171,0.800356,0.937618,0.384364,0.381571,0.383369,0.378353
2,79,3,NaN,NaN,0,080119-testi1,0,3,E04,227,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,79,4,NaN,NaN,0,080119-testi1,0,3,E04,1331,...,-0.000000,-0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,79,5,NaN,NaN,0,080119-testi1,0,3,E04,1662,...,-0.000000,-0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [3]:
# columns to group the results by
group_cols = ['Metadata_Plate', 'Metadata_Well']

# exclude these from averaging
exclude_cols = ['ImageNumber', 'ObjectNumber', 'Metadata_']

measurements = [k for k in list(df.columns.values) if not k.startswith(tuple(exclude_cols))]


# https://stackoverflow.com/questions/35401691/groupby-and-calculate-mean-but-keeping-all-columns

# create a new DataFrame with a MultiIndex consisting of the group_cols
# and a column for the mean of each column in metric_cols
aggs = df.groupby(group_cols)[measurements].mean()
# remove the metric_cols from df because we are going to replace them
# with the means in aggs
df.drop(measurements, axis=1, inplace=True)
# dedupe to leave only one row with each combination of group_cols
# in df
df.drop_duplicates(subset=group_cols, keep='last', inplace=True)
# add the mean columns from aggs into df
df = df.merge(right=aggs, right_index=True, left_on=group_cols, how='right')

outputFilename = basename + "_well" + extension
df.to_csv(os.path.join(workdir, outputFilename), index=False)

df.head()

,ImageNumber,ObjectNumber,Metadata_ChannelNumber,Metadata_FileLocation,Metadata_Frame,Metadata_Plate,Metadata_Series,Metadata_Site,Metadata_Well,AreaShape_Area,...,Texture_SumEntropy_Red_5_02,Texture_SumEntropy_Red_5_03,Texture_SumVariance_Red_5_00,Texture_SumVariance_Red_5_01,Texture_SumVariance_Red_5_02,Texture_SumVariance_Red_5_03,Texture_Variance_Red_5_00,Texture_Variance_Red_5_01,Texture_Variance_Red_5_02,Texture_Variance_Red_5_03
603,80,301,NaN,NaN,0,080119-testi1,0,4,E04,3113.887417,...,0.465212,0.452527,0.249433,0.226909,0.257121,0.231715,0.097619,0.093765,0.097642,0.095418
1057,81,454,NaN,NaN,0,080119-testi1,0,1,E05,3100.649780,...,0.241084,0.237188,0.173815,0.152867,0.171751,0.159605,0.064430,0.065585,0.064659,0.063562
